<a href="https://colab.research.google.com/github/anphantt2406/Heritage-Health-Prize/blob/main/%5BML2020%5D%5BHHP%5D%5BG4_PTThuyAn%5D_PrepareData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/HHP_release3/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/HHP_release3


# Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()

# Member process

In [ ]:
member_df = pd.read_csv("Members.csv")
#member_df.info()

### Gender - Onehot Encoding

In [ ]:
member_df['Male'] = 0
member_df['Female'] = 0
member_df['Unknown'] = 0
member_df.loc[member_df.Sex == 'M', 'Male'] = 1
member_df.loc[member_df.Sex == 'F', 'Female'] = 1
member_df.loc[member_df.Sex.isna(), 'Unknown'] = 1
member_df.drop(columns=['Sex'], inplace=True)

### Age - Replace by Mean

In [ ]:
member_df.loc[member_df.AgeAtFirstClaim == '0-9', 'AgeAtFirstClaim'] = 5
member_df.loc[member_df.AgeAtFirstClaim == '10-19', 'AgeAtFirstClaim'] = 15
member_df.loc[member_df.AgeAtFirstClaim == '20-29', 'AgeAtFirstClaim'] = 25
member_df.loc[member_df.AgeAtFirstClaim == '30-39', 'AgeAtFirstClaim'] = 35
member_df.loc[member_df.AgeAtFirstClaim == '40-49', 'AgeAtFirstClaim'] = 45
member_df.loc[member_df.AgeAtFirstClaim == '50-59', 'AgeAtFirstClaim'] = 55
member_df.loc[member_df.AgeAtFirstClaim == '60-69', 'AgeAtFirstClaim'] = 65
member_df.loc[member_df.AgeAtFirstClaim == '70-79', 'AgeAtFirstClaim'] = 75
member_df.loc[member_df.AgeAtFirstClaim == '80+', 'AgeAtFirstClaim'] = 80

#replace missing values
member_df.loc[member_df.AgeAtFirstClaim.isna(), 'AgeAtFirstClaim'] = 45 

# Claims process

In [ ]:
claims_df = pd.read_csv("3_Claims.csv")
#claims_df.info()
claims_df.columns = ['MemberID', 'Provider', 'Vendor', 'PCP', 'Year', 'Specialty',
       'Place', 'PayDelay', 'LOS', 'DSFS', 'PCG',
       'Charlson', 'Procedure', 'SupLOS']

### PayDelay

In [ ]:
claims_df.loc[claims_df['PayDelay']=='162+','PayDelay'] = 162
claims_df['PayDelay'] = claims_df['PayDelay'].astype(int)

### Length of Stay

In [ ]:
claims_df.loc[claims_df['LOS']=='1 day','LOS'] = 1
claims_df.loc[claims_df['LOS']=='2 days','LOS'] = 2
claims_df.loc[claims_df['LOS']=='3 days','LOS'] = 3
claims_df.loc[claims_df['LOS']=='4 days','LOS'] = 4
claims_df.loc[claims_df['LOS']=='5 days','LOS'] = 5
claims_df.loc[claims_df['LOS']=='6 days','LOS'] = 6
claims_df.loc[claims_df['LOS']=='1- 2 weeks','LOS'] = 11
claims_df.loc[claims_df['LOS']=='2- 4 weeks','LOS'] = 21
claims_df.loc[claims_df['LOS']=='4- 8 weeks','LOS'] = 42
claims_df.loc[claims_df['LOS']=='26+ weeks','LOS'] = 180

claims_df['LOS'].fillna(0, inplace=True)
claims_df['LOS'] = claims_df['LOS'].astype(int)

## Separate the data for 3 years

In [ ]:
claims_Y1_df = claims_df.query("Year == 'Y1'").drop(['Year'], axis=1).reset_index(drop=True) 
claims_Y2_df = claims_df.query("Year == 'Y2'").drop(['Year'], axis=1).reset_index(drop=True) 
claims_Y3_df = claims_df.query("Year == 'Y3'").drop(['Year'], axis=1).reset_index(drop=True)

##  Process data of Unique MemberID

In [ ]:
#COUNT the number of provider, vendor, claim, ... for each UNIQUE MEMBER
claims_Y1_grouped = pd.DataFrame(claims_Y1_df.groupby(['MemberID']).agg({
    'Provider': ['count','nunique'],
    'Specialty' : 'nunique',
    'PCG' : 'nunique',
    'Procedure' : 'nunique',
    'PayDelay' : 'sum',
    'LOS' : 'sum'
}).reset_index())
claims_Y1_grouped.columns = ['MemberID', 'no_Claims', 'no_Providers','no_Specialties','no_PCG',\
                             'no_Procedure','sum_PayDelay','sum_LOS']


#one-hot encoding
claims_Y1_cat = claims_Y1_df.select_dtypes(include=['object'])
cat_cols = list(claims_Y1_cat.columns)

for col in cat_cols:
  onehot_features = pd.get_dummies(claims_Y1_cat[col])
  labels = list(onehot_features.columns) 
  for i in range(len(labels)):
    col_encode = col + "_" + labels[i]
    claims_Y1_df[col_encode] = onehot_features[onehot_features.columns[i]]
  claims_Y1_df = claims_Y1_df.drop([col], axis=1)

claims_Y1_encode = claims_Y1_df.reset_index(drop=True)
claims_Y1_encode = claims_Y1_encode.drop(['Provider','Vendor','PCP','LOS','SupLOS','PayDelay'],axis =1)
claims_Y1_encode.sort_values('MemberID',ignore_index=True)


#count one-hot values base on memberID
Y1_encode_group = pd.DataFrame()
Y1_encode_group['MemberID'] = claims_Y1_grouped['MemberID']
count_unique = list(claims_Y1_encode.columns)
count_unique.remove('MemberID')
for col in count_unique:
  df = claims_Y1_encode.groupby(['MemberID'])[col].agg('sum')
  Y1_encode_group = pd.merge(Y1_encode_group, df, on=['MemberID'])


claims_Y1_processed = pd.merge(claims_Y1_grouped, Y1_encode_group, on=['MemberID'])
claims_Y1_processed

,MemberID,no_Claims,no_Providers,no_Specialties,no_PCG,no_Procedure,sum_PayDelay,sum_LOS,Specialty_Anesthesiology,Specialty_Diagnostic Imaging,Specialty_Emergency,Specialty_General Practice,Specialty_Internal,Specialty_Laboratory,Specialty_Obstetrics and Gynecology,Specialty_Other,Specialty_Pathology,Specialty_Pediatrics,Specialty_Rehabilitation,Specialty_Surgery,Place_Ambulance,Place_Home,Place_Independent Lab,Place_Inpatient Hospital,Place_Office,Place_Other,Place_Outpatient Hospital,Place_Urgent Care,DSFS_0- 1 month,DSFS_1- 2 months,DSFS_10-11 months,DSFS_11-12 months,DSFS_2- 3 months,DSFS_3- 4 months,DSFS_4- 5 months,DSFS_5- 6 months,DSFS_6- 7 months,DSFS_7- 8 months,DSFS_8- 9 months,DSFS_9-10 months,...,PCG_MSC2a3,PCG_NEUMENT,PCG_ODaBNCA,PCG_PERINTL,PCG_PERVALV,PCG_PNCRDZ,PCG_PNEUM,PCG_PRGNCY,PCG_RENAL1,PCG_RENAL2,PCG_RENAL3,PCG_RESPR4,PCG_ROAMI,PCG_SEIZURE,PCG_SEPSIS,PCG_SKNAUT,PCG_STROKE,PCG_TRAUMA,PCG_UTI,Charlson_0,Charlson_1-2,Charlson_3-4,Charlson_5+,Procedure_ANES,Procedure_EM,Procedure_MED,Procedure_PL,Procedure_RAD,Procedure_SAS,Procedure_SCS,Procedure_SDS,Procedure_SEOA,Procedure_SGS,Procedure_SIS,Procedure_SMCD,Procedure_SMS,Procedure_SNS,Procedure_SO,Procedure_SRS,Procedure_SUS
0,210,8,4,3,4,5,720,2,0,0,2,0,4,2,0,0,0,0,0,0,0,0,2,0,4,0,0,2,3,2,0,0,0,1,0,0,1,0,0,1,...,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,1,4,1,0,0,1,1,0,0,0,0,0,0,0,0,0
1,3197,5,3,2,2,2,492,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0,3,0,0,2,1,1,0,2,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,5,0,0,0,0,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3889,13,7,4,5,5,919,3,0,1,4,0,4,4,0,0,0,0,0,0,0,0,4,0,4,0,0,5,12,1,0,0,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,2,0,0,9,4,0,0,0,5,0,4,1,0,1,2,0,0,0,0,0,0,0,0,0
3,4187,4,3,3,3,2,340,0,0,1,0,0,2,0,0,1,0,0,0,0,0,0,0,0,4,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,9063,4,2,2,1,2,241,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76033,99995554,35,3,3,3,4,3899,0,0,0,0,2,26,7,0,0,0,0,0,0,0,0,7,0,28,0,0,0,1,0,0,0,2,0,1,6,7,7,4,7,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29,0,0,0,35,0,0,0,0,4,24,3,0,0,4,0,0,0,0,0,0,0,0,0,0
76034,99996214,1,1,1,1,1,19,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
76035,99997485,1,1,1,1,1,130,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
76036,99997895,14,5,4,6,4,539,0,0,1,0,0,4,7,0,0,0,0,0,2,0,0,7,0,7,0,0,0,2,1,0,0,1,2,2,0,0,0,3,1,...,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,14,0,0,0,0,6,0,4,1,0,3,0,0,0,0,0,0,0,0,0,0


In [ ]:
#COUNT the number of provider, vendor, claim, ... for each UNIQUE MEMBER
claims_Y2_grouped = pd.DataFrame(claims_Y2_df.groupby(['MemberID']).agg({
    'Provider': ['count','nunique'],
    'Specialty' : 'nunique',
    'PCG' : 'nunique',
    'Procedure' : 'nunique',
    'PayDelay' : 'sum',
    'LOS' : 'sum'
}).reset_index())
claims_Y2_grouped.columns = ['MemberID', 'no_Claims', 'no_Providers','no_Specialties','no_PCG',\
                             'no_Procedure','sum_PayDelay','sum_LOS']


#one-hot encoding
claims_Y2_cat = claims_Y2_df.select_dtypes(include=['object'])
cat_cols = list(claims_Y2_cat.columns)

for col in cat_cols:
  onehot_features = pd.get_dummies(claims_Y2_cat[col])
  labels = list(onehot_features.columns) 
  for i in range(len(labels)):
    col_encode = col + "_" + labels[i]
    claims_Y2_df[col_encode] = onehot_features[onehot_features.columns[i]]
  claims_Y2_df = claims_Y2_df.drop([col], axis=1)

claims_Y2_encode = claims_Y2_df.reset_index(drop=True)
claims_Y2_encode = claims_Y2_encode.drop(['Provider','Vendor','PCP','LOS','SupLOS','PayDelay'],axis =1)
claims_Y2_encode.sort_values('MemberID',ignore_index=True)


#count one-hot values base on memberID
Y2_encode_group = pd.DataFrame()
Y2_encode_group['MemberID'] = claims_Y2_grouped['MemberID']
count_unique = list(claims_Y2_encode.columns)
count_unique.remove('MemberID')
for col in count_unique:
  df = claims_Y2_encode.groupby(['MemberID'])[col].agg('sum')
  Y2_encode_group = pd.merge(Y2_encode_group, df, on=['MemberID'])


claims_Y2_processed = pd.merge(claims_Y2_grouped, Y2_encode_group, on=['MemberID'])
claims_Y2_processed

,MemberID,no_Claims,no_Providers,no_Specialties,no_PCG,no_Procedure,sum_PayDelay,sum_LOS,Specialty_Anesthesiology,Specialty_Diagnostic Imaging,Specialty_Emergency,Specialty_General Practice,Specialty_Internal,Specialty_Laboratory,Specialty_Obstetrics and Gynecology,Specialty_Other,Specialty_Pathology,Specialty_Pediatrics,Specialty_Rehabilitation,Specialty_Surgery,Place_Ambulance,Place_Home,Place_Independent Lab,Place_Inpatient Hospital,Place_Office,Place_Other,Place_Outpatient Hospital,Place_Urgent Care,DSFS_0- 1 month,DSFS_1- 2 months,DSFS_10-11 months,DSFS_11-12 months,DSFS_2- 3 months,DSFS_3- 4 months,DSFS_4- 5 months,DSFS_5- 6 months,DSFS_6- 7 months,DSFS_7- 8 months,DSFS_8- 9 months,DSFS_9-10 months,...,PCG_MSC2a3,PCG_NEUMENT,PCG_ODaBNCA,PCG_PERINTL,PCG_PERVALV,PCG_PNCRDZ,PCG_PNEUM,PCG_PRGNCY,PCG_RENAL1,PCG_RENAL2,PCG_RENAL3,PCG_RESPR4,PCG_ROAMI,PCG_SEIZURE,PCG_SEPSIS,PCG_SKNAUT,PCG_STROKE,PCG_TRAUMA,PCG_UTI,Charlson_0,Charlson_1-2,Charlson_3-4,Charlson_5+,Procedure_ANES,Procedure_EM,Procedure_MED,Procedure_PL,Procedure_RAD,Procedure_SAS,Procedure_SCS,Procedure_SDS,Procedure_SEOA,Procedure_SGS,Procedure_SIS,Procedure_SMCD,Procedure_SMS,Procedure_SNS,Procedure_SO,Procedure_SRS,Procedure_SUS
0,4,1,1,1,1,1,43,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,210,6,3,3,2,3,308,0,0,0,0,0,1,1,0,4,0,0,0,0,0,0,1,0,5,0,0,0,2,0,0,0,0,2,0,0,1,0,0,1,...,1,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3197,5,4,4,2,4,148,0,0,1,0,0,1,2,0,0,0,1,0,0,0,0,2,0,3,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,5,0,0,0,0,2,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0
3,3457,1,1,1,1,1,63,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3713,10,5,3,5,3,562,0,0,1,0,7,0,2,0,0,0,0,0,0,0,0,2,0,8,0,0,0,1,5,0,0,0,0,0,0,0,0,4,0,...,2,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,2,10,0,0,0,0,5,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71430,99985996,5,2,2,2,2,316,0,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,2,1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71431,99987030,4,3,2,1,3,125,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
71432,99995391,11,2,2,4,4,533,0,0,0,0,0,6,0,0,0,0,0,0,5,0,0,0,0,11,0,0,0,2,1,0,0,4,2,0,2,0,0,0,0,...,0,5,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,11,0,0,0,0,3,6,1,1,0,0,0,0,0,0,0,0,0,0,0,0
71433,99995554,15,3,3,3,4,581,0,0,0,0,2,11,2,0,0,0,0,0,0,0,0,2,0,13,0,0,0,10,2,0,0,1,2,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,11,0,0,0,15,0,0,0,0,2,11,1,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
#COUNT the number of provider, vendor, claim, ... for each UNIQUE MEMBER
claims_Y3_grouped = pd.DataFrame(claims_Y3_df.groupby(['MemberID']).agg({
    'Provider': ['count','nunique'],
    'Specialty' : 'nunique',
    'PCG' : 'nunique',
    'Procedure' : 'nunique',
    'PayDelay' : 'sum',
    'LOS' : 'sum'
}).reset_index())
claims_Y3_grouped.columns = ['MemberID', 'no_Claims', 'no_Providers','no_Specialties','no_PCG',\
                             'no_Procedure','sum_PayDelay','sum_LOS']


#one-hot encoding
claims_Y3_cat = claims_Y3_df.select_dtypes(include=['object'])
cat_cols = list(claims_Y3_cat.columns)

for col in cat_cols:
  onehot_features = pd.get_dummies(claims_Y3_cat[col])
  labels = list(onehot_features.columns) 
  for i in range(len(labels)):
    col_encode = col + "_" + labels[i]
    claims_Y3_df[col_encode] = onehot_features[onehot_features.columns[i]]
  claims_Y3_df = claims_Y3_df.drop([col], axis=1)

claims_Y3_encode = claims_Y3_df.reset_index(drop=True)
claims_Y3_encode = claims_Y3_encode.drop(['Provider','Vendor','PCP','LOS','SupLOS','PayDelay'],axis =1)
claims_Y3_encode.sort_values('MemberID',ignore_index=True)


#count one-hot values base on memberID
Y3_encode_group = pd.DataFrame()
Y3_encode_group['MemberID'] = claims_Y3_grouped['MemberID']
count_unique = list(claims_Y3_encode.columns)
count_unique.remove('MemberID')
for col in count_unique:
  df = claims_Y3_encode.groupby(['MemberID'])[col].agg('sum')
  Y3_encode_group = pd.merge(Y3_encode_group, df, on=['MemberID'])


claims_Y3_processed = pd.merge(claims_Y3_grouped, Y3_encode_group, on=['MemberID'])
claims_Y3_processed

,MemberID,no_Claims,no_Providers,no_Specialties,no_PCG,no_Procedure,sum_PayDelay,sum_LOS,Specialty_Anesthesiology,Specialty_Diagnostic Imaging,Specialty_Emergency,Specialty_General Practice,Specialty_Internal,Specialty_Laboratory,Specialty_Obstetrics and Gynecology,Specialty_Other,Specialty_Pathology,Specialty_Pediatrics,Specialty_Rehabilitation,Specialty_Surgery,Place_Ambulance,Place_Home,Place_Independent Lab,Place_Inpatient Hospital,Place_Office,Place_Other,Place_Outpatient Hospital,Place_Urgent Care,DSFS_0- 1 month,DSFS_1- 2 months,DSFS_10-11 months,DSFS_11-12 months,DSFS_2- 3 months,DSFS_3- 4 months,DSFS_4- 5 months,DSFS_5- 6 months,DSFS_6- 7 months,DSFS_7- 8 months,DSFS_8- 9 months,DSFS_9-10 months,...,PCG_MSC2a3,PCG_NEUMENT,PCG_ODaBNCA,PCG_PERINTL,PCG_PERVALV,PCG_PNCRDZ,PCG_PNEUM,PCG_PRGNCY,PCG_RENAL1,PCG_RENAL2,PCG_RENAL3,PCG_RESPR4,PCG_ROAMI,PCG_SEIZURE,PCG_SEPSIS,PCG_SKNAUT,PCG_STROKE,PCG_TRAUMA,PCG_UTI,Charlson_0,Charlson_1-2,Charlson_3-4,Charlson_5+,Procedure_ANES,Procedure_EM,Procedure_MED,Procedure_PL,Procedure_RAD,Procedure_SAS,Procedure_SCS,Procedure_SDS,Procedure_SEOA,Procedure_SGS,Procedure_SIS,Procedure_SMCD,Procedure_SMS,Procedure_SNS,Procedure_SO,Procedure_SRS,Procedure_SUS
0,210,4,2,2,2,2,143,0,0,0,0,0,0,1,0,3,0,0,0,0,0,0,1,0,3,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3197,11,3,2,3,2,379,0,0,0,0,0,9,0,0,0,0,2,0,0,0,0,0,0,11,0,0,0,5,1,1,0,1,0,0,1,0,2,0,0,...,2,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,11,0,0,0,8,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4048,21,10,6,5,7,675,3,0,1,2,0,7,4,0,0,1,0,0,7,3,0,4,0,10,0,2,3,6,11,0,0,5,0,0,0,0,0,0,0,...,9,0,0,0,0,0,0,0,0,0,10,1,0,0,0,0,0,0,1,13,9,0,0,0,7,2,5,2,0,2,3,0,1,0,0,0,0,0,0,0
3,5187,13,4,4,4,4,563,0,0,2,0,2,1,0,0,0,0,8,0,0,0,0,0,0,11,0,0,2,5,0,0,0,5,2,1,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,11,2,0,0,0,6,2,1,4,0,0,0,0,0,0,0,0,0,0,0,0
4,14033,8,7,6,2,6,330,2,1,1,3,0,1,0,0,0,1,0,0,1,0,0,0,5,0,0,0,3,8,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,1,2,0,2,1,0,0,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70937,99987030,8,3,3,3,4,0,0,0,1,0,0,3,4,0,0,0,0,0,0,0,0,4,0,4,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,...,6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,3,0,2,1,0,2,0,0,0,0,0,0,0,0,0,0
70938,99992565,25,2,2,2,6,536,0,0,0,0,22,0,3,0,0,0,0,0,0,0,0,3,0,22,0,0,0,13,0,1,0,3,3,0,0,1,0,4,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,0,0,11,5,3,1,0,4,0,0,0,0,0,1,0,0,0,0
70939,99993334,2,1,1,1,2,44,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
70940,99995391,34,10,6,11,7,794,0,1,1,0,2,19,6,0,0,0,0,0,5,3,0,6,0,25,0,0,0,7,3,0,3,1,3,3,1,1,5,7,0,...,5,6,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,34,0,0,0,1,15,7,5,1,0,3,2,0,0,0,0,0,0,0,0,0


# Drug Count & Lab Count

In [ ]:
drug_df = pd.read_csv("DrugCount.csv")
# Replace DrugCount 7+ with 7
drug_df.loc[drug_df['DrugCount']=='7+','DrugCount'] = 7
drug_df['DrugCount'] = drug_df['DrugCount'].astype(int)
drug_df = drug_df.drop(['DSFS'], axis = 1)

lab_df = pd.read_csv("LabCount.csv")
# Replace LabCount 10+ with 10
lab_df.loc[lab_df['LabCount']=='10+', 'LabCount'] = 10
lab_df['LabCount'] = lab_df['LabCount'].astype(int)
lab_df = lab_df.drop(['DSFS'], axis = 1)


drug_Y1_df = drug_df.query("Year == 'Y1'").drop(['Year'], axis = 1).reset_index(drop=True) 
drug_Y2_df = drug_df.query("Year == 'Y2'").drop(['Year'], axis = 1).reset_index(drop=True) 

lab_Y1_df = lab_df.query("Year == 'Y1'").drop(['Year'], axis = 1).reset_index(drop=True) 
lab_Y2_df = lab_df.query("Year == 'Y2'").drop(['Year'], axis = 1).reset_index(drop=True) 

In [ ]:
drug_Y1_df.sort_values('MemberID', ignore_index=True)
drug_Y1_group = pd.DataFrame(drug_Y1_df.groupby(['MemberID'])['DrugCount'].agg('sum').reset_index())
drug_Y1_group.columns = ['MemberID','DrugSum']

drug_Y1 = pd.DataFrame()
drug_Y1['MemberID'] = claims_Y1_grouped['MemberID'].astype(int)
drug_Y1['DrugSum'] = 0
drug_Y1['DrugSum'] = drug_Y1['DrugSum'].astype(int)

for id in drug_Y1_group.MemberID:
  drug_Y1.drop(drug_Y1.index[drug_Y1['MemberID'] == id], inplace = True)
drug_Y1 = drug_Y1.append(drug_Y1_group)
drug_Y1.sort_values('MemberID', ignore_index=True)

,MemberID,DrugSum
0,210,5
1,3197,5
2,3889,30
3,4187,61
4,9063,2
...,...,...
76033,99995554,8
76034,99996214,0
76035,99997485,0
76036,99997895,0


In [ ]:
drug_Y2_df.sort_values('MemberID', ignore_index=True)
drug_Y2_group = pd.DataFrame(drug_Y2_df.groupby(['MemberID'])['DrugCount'].agg('sum').reset_index())
drug_Y2_group.columns = ['MemberID','DrugSum']

drug_Y2 = pd.DataFrame()
drug_Y2['MemberID'] = claims_Y2_grouped['MemberID'].astype(int)
drug_Y2['DrugSum'] = 0
drug_Y2['DrugSum'] = drug_Y2['DrugSum'].astype(int)

for id in drug_Y2_group.MemberID:
  drug_Y2.drop(drug_Y2.index[drug_Y2['MemberID'] == id], inplace = True)
drug_Y2 = drug_Y2.append(drug_Y2_group)
drug_Y2.sort_values('MemberID', ignore_index=True)

,MemberID,DrugSum
0,4,0
1,210,0
2,3197,3
3,3457,0
4,3713,17
...,...,...
71430,99985996,18
71431,99987030,2
71432,99995391,14
71433,99995554,2


In [ ]:
lab_Y1_df.sort_values('MemberID', ignore_index=True)
lab_Y1_group = pd.DataFrame(lab_Y1_df.groupby(['MemberID'])['LabCount'].agg('sum').reset_index())
lab_Y1_group.columns = ['MemberID','LabSum']

lab_Y1 = pd.DataFrame()
lab_Y1['MemberID'] = claims_Y1_grouped['MemberID'].astype(int)
lab_Y1['LabSum'] = 0
lab_Y1['LabSum'] = lab_Y1['LabSum'].astype(int)

for id in lab_Y1_group.MemberID:
  lab_Y1.drop(lab_Y1.index[lab_Y1['MemberID'] == id], inplace = True)
lab_Y1 = lab_Y1.append(lab_Y1_group)
lab_Y1.sort_values('MemberID', ignore_index=True)

,MemberID,LabSum
0,210,2
1,3197,0
2,3889,10
3,4187,0
4,9063,0
...,...,...
76033,99995554,11
76034,99996214,0
76035,99997485,0
76036,99997895,10


In [ ]:
lab_Y2_df.sort_values('MemberID', ignore_index=True)
lab_Y2_group = pd.DataFrame(lab_Y2_df.groupby(['MemberID'])['LabCount'].agg('sum').reset_index())
lab_Y2_group.columns = ['MemberID','LabSum']

lab_Y2 = pd.DataFrame()
lab_Y2['MemberID'] = claims_Y2_grouped['MemberID'].astype(int)
lab_Y2['LabSum'] = 0
lab_Y2['LabSum'] = lab_Y2['LabSum'].astype(int)

for id in lab_Y2_group.MemberID:
  lab_Y2.drop(lab_Y2.index[lab_Y2['MemberID'] == id], inplace = True)
lab_Y2 = lab_Y2.append(lab_Y2_group)
lab_Y2.sort_values('MemberID', ignore_index=True)

,MemberID,LabSum
0,4,0
1,210,1
2,3197,2
3,3457,0
4,3713,9
...,...,...
71430,99985996,0
71431,99987030,0
71432,99995391,1
71433,99995554,10


In [ ]:
labdrug_Y1 = pd.merge(drug_Y1, lab_Y1, on=['MemberID'], how='outer')
labdrug_Y1 = labdrug_Y1.sort_values('MemberID', ignore_index=True)

labdrug_Y2 = pd.merge(drug_Y2, lab_Y2, on=['MemberID'], how='outer')
labdrug_Y2 = labdrug_Y2.sort_values('MemberID', ignore_index=True)

# Targets Observation

In [ ]:
targetY1_DIHY2_df = pd.read_csv("DaysInHospital_Y2.csv")
targetY1_DIHY2_df['CLASS'] = -1
targetY1_DIHY2_df.loc[targetY1_DIHY2_df.DaysInHospital != 0, 'CLASS'] = 1
targetY1_DIHY2_df.drop(columns=['ClaimsTruncated'], inplace=True)

targetY1_DIHY2_df.CLASS.value_counts()

-1    64269
 1    11769
Name: CLASS, dtype: int64

In [ ]:
targetY2_DIHY3_df = pd.read_csv("DaysInHospital_Y3.csv")
targetY2_DIHY3_df['CLASS'] = -1
targetY2_DIHY3_df.loc[targetY2_DIHY3_df.DaysInHospital != 0, 'CLASS'] = 1
targetY2_DIHY3_df.drop(columns=['ClaimsTruncated'], inplace=True)

targetY2_DIHY3_df.CLASS.value_counts()

-1    60706
 1    10729
Name: CLASS, dtype: int64

# Save the processed data

In [ ]:
%cd /content/drive/My Drive/HHP_release3/new_data/

/content/drive/My Drive/HHP_release3/new_data


In [ ]:
Y1_data = pd.merge(claims_Y1_processed, labdrug_Y1, on=['MemberID'])
Y2_data = pd.merge(claims_Y2_processed, labdrug_Y2, on=['MemberID'])

In [ ]:
Y1_fulldata = pd.merge(Y1_data, targetY1_DIHY2_df, on=['MemberID'])
Y1_fulldata.rename(columns={"DaysInHospital": "TARGET"}, inplace=True)
Y1_fulldata

,MemberID,no_Claims,no_Providers,no_Specialties,no_PCG,no_Procedure,sum_PayDelay,sum_LOS,Specialty_Anesthesiology,Specialty_Diagnostic Imaging,Specialty_Emergency,Specialty_General Practice,Specialty_Internal,Specialty_Laboratory,Specialty_Obstetrics and Gynecology,Specialty_Other,Specialty_Pathology,Specialty_Pediatrics,Specialty_Rehabilitation,Specialty_Surgery,Place_Ambulance,Place_Home,Place_Independent Lab,Place_Inpatient Hospital,Place_Office,Place_Other,Place_Outpatient Hospital,Place_Urgent Care,DSFS_0- 1 month,DSFS_1- 2 months,DSFS_10-11 months,DSFS_11-12 months,DSFS_2- 3 months,DSFS_3- 4 months,DSFS_4- 5 months,DSFS_5- 6 months,DSFS_6- 7 months,DSFS_7- 8 months,DSFS_8- 9 months,DSFS_9-10 months,...,PCG_PERVALV,PCG_PNCRDZ,PCG_PNEUM,PCG_PRGNCY,PCG_RENAL1,PCG_RENAL2,PCG_RENAL3,PCG_RESPR4,PCG_ROAMI,PCG_SEIZURE,PCG_SEPSIS,PCG_SKNAUT,PCG_STROKE,PCG_TRAUMA,PCG_UTI,Charlson_0,Charlson_1-2,Charlson_3-4,Charlson_5+,Procedure_ANES,Procedure_EM,Procedure_MED,Procedure_PL,Procedure_RAD,Procedure_SAS,Procedure_SCS,Procedure_SDS,Procedure_SEOA,Procedure_SGS,Procedure_SIS,Procedure_SMCD,Procedure_SMS,Procedure_SNS,Procedure_SO,Procedure_SRS,Procedure_SUS,DrugSum,LabSum,TARGET,CLASS
0,210,8,4,3,4,5,720,2,0,0,2,0,4,2,0,0,0,0,0,0,0,0,2,0,4,0,0,2,3,2,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,1,4,1,0,0,1,1,0,0,0,0,0,0,0,0,0,5,2,0,-1
1,3197,5,3,2,2,2,492,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0,3,0,0,2,1,1,0,2,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,5,0,0,0,0,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5,0,0,-1
2,3889,13,7,4,5,5,919,3,0,1,4,0,4,4,0,0,0,0,0,0,0,0,4,0,4,0,0,5,12,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4,0,0,2,0,0,9,4,0,0,0,5,0,4,1,0,1,2,0,0,0,0,0,0,0,0,0,30,10,0,-1
3,4187,4,3,3,3,2,340,0,0,1,0,0,2,0,0,1,0,0,0,0,0,0,0,0,4,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,61,0,0,-1
4,9063,4,2,2,1,2,241,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76033,99995554,35,3,3,3,4,3899,0,0,0,0,2,26,7,0,0,0,0,0,0,0,0,7,0,28,0,0,0,1,0,0,0,2,0,1,6,7,7,4,7,...,0,0,0,0,0,0,0,0,0,0,0,29,0,0,0,35,0,0,0,0,4,24,3,0,0,4,0,0,0,0,0,0,0,0,0,0,8,11,0,-1
76034,99996214,1,1,1,1,1,19,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
76035,99997485,1,1,1,1,1,130,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
76036,99997895,14,5,4,6,4,539,0,0,1,0,0,4,7,0,0,0,0,0,2,0,0,7,0,7,0,0,0,2,1,0,0,1,2,2,0,0,0,3,1,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,14,0,0,0,0,6,0,4,1,0,3,0,0,0,0,0,0,0,0,0,0,0,10,0,-1


In [ ]:
Y2_fulldata = pd.merge(Y2_data, targetY2_DIHY3_df, on=['MemberID'])
Y2_fulldata.rename(columns={"DaysInHospital": "TARGET"}, inplace=True)
Y2_fulldata

,MemberID,no_Claims,no_Providers,no_Specialties,no_PCG,no_Procedure,sum_PayDelay,sum_LOS,Specialty_Anesthesiology,Specialty_Diagnostic Imaging,Specialty_Emergency,Specialty_General Practice,Specialty_Internal,Specialty_Laboratory,Specialty_Obstetrics and Gynecology,Specialty_Other,Specialty_Pathology,Specialty_Pediatrics,Specialty_Rehabilitation,Specialty_Surgery,Place_Ambulance,Place_Home,Place_Independent Lab,Place_Inpatient Hospital,Place_Office,Place_Other,Place_Outpatient Hospital,Place_Urgent Care,DSFS_0- 1 month,DSFS_1- 2 months,DSFS_10-11 months,DSFS_11-12 months,DSFS_2- 3 months,DSFS_3- 4 months,DSFS_4- 5 months,DSFS_5- 6 months,DSFS_6- 7 months,DSFS_7- 8 months,DSFS_8- 9 months,DSFS_9-10 months,...,PCG_PERVALV,PCG_PNCRDZ,PCG_PNEUM,PCG_PRGNCY,PCG_RENAL1,PCG_RENAL2,PCG_RENAL3,PCG_RESPR4,PCG_ROAMI,PCG_SEIZURE,PCG_SEPSIS,PCG_SKNAUT,PCG_STROKE,PCG_TRAUMA,PCG_UTI,Charlson_0,Charlson_1-2,Charlson_3-4,Charlson_5+,Procedure_ANES,Procedure_EM,Procedure_MED,Procedure_PL,Procedure_RAD,Procedure_SAS,Procedure_SCS,Procedure_SDS,Procedure_SEOA,Procedure_SGS,Procedure_SIS,Procedure_SMCD,Procedure_SMS,Procedure_SNS,Procedure_SO,Procedure_SRS,Procedure_SUS,DrugSum,LabSum,TARGET,CLASS
0,4,1,1,1,1,1,43,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
1,210,6,3,3,2,3,308,0,0,0,0,0,1,1,0,4,0,0,0,0,0,0,1,0,5,0,0,0,2,0,0,0,0,2,0,0,1,0,0,1,...,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-1
2,3197,5,4,4,2,4,148,0,0,1,0,0,1,2,0,0,0,1,0,0,0,0,2,0,3,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,5,0,0,0,0,2,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,3,2,0,-1
3,3457,1,1,1,1,1,63,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
4,3713,10,5,3,5,3,562,0,0,1,0,7,0,2,0,0,0,0,0,0,0,0,2,0,8,0,0,0,1,5,0,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,2,1,0,0,0,0,0,0,2,10,0,0,0,0,5,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,17,9,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71430,99985996,5,2,2,2,2,316,0,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,5,0,0,0,2,1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18,0,0,-1
71431,99987030,4,3,2,1,3,125,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,-1
71432,99995391,11,2,2,4,4,533,0,0,0,0,0,6,0,0,0,0,0,0,5,0,0,0,0,11,0,0,0,2,1,0,0,4,2,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,11,0,0,0,0,3,6,1,1,0,0,0,0,0,0,0,0,0,0,0,0,14,1,0,-1
71433,99995554,15,3,3,3,4,581,0,0,0,0,2,11,2,0,0,0,0,0,0,0,0,2,0,13,0,0,0,10,2,0,0,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0,0,11,0,0,0,15,0,0,0,0,2,11,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2,10,0,-1


In [ ]:
memberY1_df = pd.DataFrame()
for id in Y1_data.MemberID:
  memberY1_df = memberY1_df.append(member_df.loc[member_df.MemberID == id])

memberY2_df = pd.DataFrame()
for id in Y2_data.MemberID:
  memberY2_df = memberY2_df.append(member_df.loc[member_df.MemberID == id])

In [ ]:
Y1_df = pd.merge(memberY1_df, Y1_fulldata, on=['MemberID']).to_csv('fulldataY1.csv', index=False)
Y2_df = pd.merge(memberY2_df, Y2_fulldata, on=['MemberID']).to_csv('fulldataY2.csv', index=False)

In [ ]:
member_df.sort_values('MemberID').to_csv('Member_processed.csv', index=False)

In [ ]:
claims_Y1_processed.sort_values('MemberID').to_csv('ClaimsY1_data.csv', index=False)
claims_Y2_processed.sort_values('MemberID').to_csv('ClaimsY2_data.csv', index=False)

In [ ]:
labdrug_Y1.sort_values('MemberID').to_csv('LabDrugY1.csv', index=False)
labdrug_Y2.sort_values('MemberID').to_csv('LabDrugY2.csv', index=False)

In [ ]:
targetY1_DIHY2_df.sort_values('MemberID').to_csv('DaysInHos_Y1.csv', index=False)
targetY2_DIHY3_df.sort_values('MemberID').to_csv('DaysInHos_Y2.csv', index=False)

In [ ]:
memberY1_df.sort_values('MemberID').to_csv('memberY1.csv', index=False)
memberY2_df.sort_values('MemberID').to_csv('memberY2.csv', index=False)